In [ ]:
import json
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import re

def extract_float(value):
  
    match = re.search(r"[-+]?\d*\.\d+|\d+", value)
    if match:
        return float(match.group())
    else:
        return 0  

def caculate_score_LLM(file_path):
    labels = []
    predicts = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                label = data.get('label')  
                predict = data.get('predict')  
                predict = predict.replace(" ", ".")
                label = extract_float(label)
                predict = extract_float(predict)
                if label is not None:
                    labels.append(float(label))
                if predict is not None:
                    predicts.append(float(predict))
            except json.JSONDecodeError:
                print("Error decoding JSON line:", line)
 
    labels = pd.Series(labels).tolist()
    predicts = pd.Series(predicts).tolist()
    labels = np.array(labels).reshape(-1, 1)
    predicts = np.array(predicts).reshape(-1, 1)
    scaler1 = StandardScaler()
    scaler1.fit(labels)
    label_tr = scaler1.transform(labels)
    scaler2 = StandardScaler()
    scaler2.fit(predicts)
    predicts_tr = scaler2.transform(predicts)
    R2 = r2_score(label_tr, predicts_tr)
    RMSE = np.sqrt(mean_squared_error(label_tr, predicts_tr))  
    return R2, RMSE

file_path = r"C:\Users\Windows\Desktop\fsdownload\no_unit_predict_train\generated_predictions.jsonl"
R2,RMSE = caculate_score_LLM(file_path)
print(R2,RMSE)

0.9382991081403148 0.2483966422069453
